In [1]:
import cv2
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D,GlobalMaxPooling2D
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix,classification_report

## Augment the Data using ImageDataGenerator

In [2]:
train_val_datagen = ImageDataGenerator( 
  rotation_range=20,
  width_shift_range=0.1,
  height_shift_range=0.1,
  shear_range=0.1,
  zoom_range=0.1,
  horizontal_flip=True,
  vertical_flip=True,
  rescale=1./255,  
  validation_split=0.2,         
#   preprocessing_function=keras.applications.vgg16.preprocess_input
  )

test_datagen= ImageDataGenerator(
  rescale=1./255)

## Load Dataset

In [6]:
#train and test dataset paths
train_path='dataset/train'  
test_path='dataset/test'

train_generator = train_val_datagen.flow_from_directory(train_path, 
                                       target_size=[224,224],
                                       color_mode='rgb', 
                                       class_mode="categorical", 
                                       batch_size = 32, 
                                       shuffle=True,
                                       subset='training') 

validation_generator = train_val_datagen.flow_from_directory(train_path, 
                                       target_size=[224,224],
                                       color_mode='rgb', 
                                       class_mode="categorical",
                                       batch_size = 32,
                                       shuffle=True,
                                       subset='validation') 

test_generator = test_datagen.flow_from_directory(test_path, 
                                      target_size=[224,224],                                    
                                      color_mode='rgb', 
                                      class_mode="categorical",
                                      batch_size = 32) 

Found 896 images belonging to 7 classes.
Found 222 images belonging to 7 classes.
Found 88 images belonging to 7 classes.


In [7]:
train_generator.class_indices

{'.ipynb_checkpoints': 0,
 'freshapples': 1,
 'freshbanana': 2,
 'freshoranges': 3,
 'rottenapples': 4,
 'rottenbanana': 5,
 'rottenoranges': 6}

In [8]:
validation_generator.class_indices

{'.ipynb_checkpoints': 0,
 'freshapples': 1,
 'freshbanana': 2,
 'freshoranges': 3,
 'rottenapples': 4,
 'rottenbanana': 5,
 'rottenoranges': 6}

In [9]:
test_generator.class_indices

{'.ipynb_checkpoints': 0,
 'freshapples': 1,
 'freshbanana': 2,
 'freshoranges': 3,
 'rottenapples': 4,
 'rottenbanana': 5,
 'rottenoranges': 6}

# **Building The Deep Model Using Cnn**

In [10]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models


model0 = keras.models.Sequential()

model0.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model0.add(layers.MaxPooling2D((2, 2)))

model0.add(layers.Conv2D(64, (3, 3), activation='relu'))
model0.add(layers.MaxPooling2D((2, 2)))

model0.add(layers.Conv2D(128, (3, 3), activation='relu'))
model0.add(layers.MaxPooling2D((2, 2)))

model0.add(layers.Conv2D(128, (3, 3), activation='relu'))
model0.add(layers.MaxPooling2D((2, 2)))

model0.add(layers.Flatten())

model0.add(layers.Dense(512, activation='relu'))
model0.add(layers.Dense(6,activation='softmax'))

In [11]:
model0.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [12]:
model0.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Training the cnn model with 20 epochs

In [16]:
history0 = model0.fit(train_generator,
          validation_data=test_generator,
          steps_per_epoch=train_generator.samples/train_generator.batch_size, 
          validation_steps=test_generator.samples/test_generator.batch_size, 
          epochs=20)

Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 728, in start
      self.io_loop.start()
    File "C:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site

# Plotting Training and Validation **Accuracy**


In [ ]:
plt.plot(history0.history['accuracy'])
plt.plot(history0.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Acc','Val'], loc = 'upper left')

# Plotting Training and Validation **Error**

In [ ]:
plt.plot(history0.history['loss'])
plt.plot(history0.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['loss','Val'], loc = 'upper left')

# Evaluating the model on test data (bad loss result given)

In [ ]:
result0=model0.evaluate(test_generator, steps=test_generator.samples/test_generator.batch_size)
print(result0)
print('Error on testing dataset: ', result0[0])
print('Accuracy on testing dataset: ', result0[1])

## Saving the model

In [ ]:
model0.save('/kaggle/working/rottenvsfresh_cnn_model.h5')

## Loading the model

In [ ]:
from keras.models import load_model
model0=load_model('../input/rottenvsfresh-fruits-model/rottenvsfresh_cnn_model.h5')

# **Building The Deep Model Using Pretrained VGG16 Model**

## Load VGG16 Model with ImageNet weights
#### And also remove the last layer of the model.
### Transfer Learning

In [ ]:
vgg16_model=keras.applications.VGG16(weights='imagenet',input_shape=(224, 224, 3),include_top=False) 
#input_shape=(224,224,3),pictures of dataset are in color so they have three dimensions:height,width,number of channels which are three (red,green,blue)

# The 5 blocks of VGG16 Pretrained Model

In [ ]:
vgg16_model.summary()

# Adding layers to the pretrained VGG16 model
#### the last dense layer has the same number of neurons of the classes in the dataset.
### Freeze the model


In [ ]:
vgg16_model.trainable = False

In [ ]:
# Create inputs with correct shape
inputs = vgg16_model.input

# Call the VGG16 model
x = vgg16_model(inputs,training=False)

#pooling layer 
x = keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer with softmax as activation function
outputs = keras.layers.Dense(6, activation = 'softmax')(x)

# Combine inputs and outputs to create model
model1 = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model1.summary()

## Compile Model with categorical_crossentropy as loss function and adam optimizer

In [ ]:
model1.compile(loss='categorical_crossentropy',metrics=['accuracy'], optimizer='adam')   

## Training the Model with 20 epochs

In [ ]:
history1 = model1.fit(total_train_generator,
          validation_data=test_generator,
          steps_per_epoch=total_train_generator.samples/total_train_generator.batch_size, 
          validation_steps=test_generator.samples/test_generator.batch_size, 
          epochs=20)

# Plotting Training and Validation **Accuracy**

In [ ]:
plt.plot(history1.history['accuracy'])
plt.plot(history1.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['Acc','Val'], loc = 'upper left')

# Plotting Training and Validation **Error**

In [ ]:
plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['loss','Val'], loc = 'upper left')

# Evaluating the model on the testing dataset

In [ ]:
result1=model1.evaluate(test_generator, steps=test_generator.samples/test_generator.batch_size)
print(result1)
print('Error on testing dataset: ', result1[0])
print('Accuracy on testing dataset: ', result1[1])

## Saving the model

In [ ]:
model1.save('/kaggle/working/rottenvsfresh_vgg16_model.h5')

## Loading the model (the best model)

In [ ]:
from keras.models import load_model
model1=load_model('../input/rottenvsfresh-fruits-model/rottenvsfresh_vgg16_model.h5')

# Testing the final model on a new photo from the testing dataset

In [ ]:
names = [fresh_apples_test_dir,
         fresh_banana_test_dir,
         fresh_oranges_test_dir,
         rotten_apples_test_dir,
         rotten_banana_test_dir,
         rotten_oranges_test_dir
]

def load_rand_test_image():
    X=[]
    dir_path='/kaggle/input/fruits-fresh-and-rotten-for-classification/dataset/test'
    for sub_dir in tqdm(os.listdir(dir_path)):
        path_main=os.path.join(dir_path,sub_dir)
        i=0
        for img_name in os.listdir(path_main):
            if i>=5:
                break
            img=cv2.imread(os.path.join(path_main,img_name))
            img=cv2.resize(img,(224,224))
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            X.append(img)
            i+=1
    return X

def show_subpot(X,title=False,Y=None):
    if X.shape[0]==30:
        f, ax= plt.subplots(6,5, figsize=(40,60))
        list_fruits=['rottenbanana','freshoranges','rottenoranges', 'freshbanana', 'rottenapples',   'freshapples']
        for i,img in enumerate(X):
            ax[i//5][i%5].imshow(img, aspect='auto')
            if title==False:
                ax[i//5][i%5].set_title(list_fruits[i//5])
            elif title and Y is not None:
                ax[i//5][i%5].set_title(Y[i])
        plt.show()
    else:
        print('Cannot plot')
        
test_images_arr=np.array(load_rand_test_image())
test_images_arr.shape

In [ ]:
show_subpot(test_images_arr) 

In [ ]:
# class mapping to indices by the data generator
# {'freshapples': 0,
#  'freshbanana': 1,
#  'freshoranges': 2,
#  'rottenapples': 3,
#  'rottenbanana': 4,
#  'rottenoranges': 5}

#Change the model name here
classes = model1.predict(test_images_arr, batch_size=32) #model1
img_acc=[]
for i,cl_pr in enumerate(classes):
    
    class_ind=np.argmax(cl_pr)
    img_acc.append(class_ind)

    prediction = ''

    if class_ind==0:
        prediction = 'fresh apple'
    elif class_ind==1:
        prediction = 'fresh banana'
    elif class_ind==2:
        prediction = 'fresh orange'
    elif class_ind==3:
        prediction = 'rotten apple'
    elif class_ind==4:
        prediction = 'rotten banana'
    elif class_ind==5:
        prediction = 'rotten orange'

    print("The predicted class on image number ",i+1," is class number ",class_ind," and the class name ",prediction)
    print("......................................................................")

print("The Correct labels of the images :","\n 4 4 4 4 4 2 2 2 2 2 5 5 5 5 5 1 1 1 1 1 3 3 3 3 3 0 0 0 0 0")
print("The predicted labels of the images :",img_acc)

In [ ]:
names = [fresh_apples_test_dir,
         fresh_banana_test_dir,
         fresh_oranges_test_dir,
         rotten_apples_test_dir,
         rotten_banana_test_dir,
         rotten_oranges_test_dir
]

name_rand = random.choice(names)
filename = os.listdir(name_rand)
sample = random.choice(filename)
fn = os.path.join(name_rand,sample)
print("the path of the photo is ",fn)


img = load_img(fn, target_size=(224,224))
print(type(img))
plt.imshow(img)


x = img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

#change the model here
classes = model1.predict(images, batch_size=32)
class_ind=np.argmax(classes[0])
print("class predicted by the model",class_ind)


prediction = ''

if class_ind==0:
    prediction = 'fresh apple'
elif class_ind==1:
    prediction = 'fresh banana'
elif class_ind==2:
    prediction = 'fresh orange'
elif class_ind==3:
    prediction = 'rotten apple'
elif class_ind==4:
    prediction = 'rotten banana'
elif class_ind==5:
    prediction = 'rotten orange'

print("The name of the predicted class:",prediction)